In [1]:
#Simulation manual:
# Initalize the following objects:
    #CPG: Uses OR for EVT and EVT*core_vol to generate noEVT arm from EVT arm
    #M: Simulates mRS, age, and sex related mortality in the long run
    #RS: Simulates mRS, years since index stroke, and age related mRS decay due to recurrent stroke
    #C: Contains and computes costs per mRS per follow-up year, and costs of intial treatment/diagnostics
    #Q: Contains and computes QALYs per mRS per follow-up year
    #Sim: Combines the above objects in a simulator that runs simulations when called
    #df: Observed 90d mRS and baseline variables of all patients in EVT arm
#Baseline simulation:
    #Select IDs and pass df to run simulate_IDs -->
    #returns totals_res, extend_res
    #Running cohort_outcome_per_threshold will 
    #return:
        #out: NMB, d_costs, d_qalys per ID
        #aggr: average ICER, NMB, d_costs, d_qalys between strategies over the baseline cohort
#Probabilistic sensitivity:
    #Run probabilistic_simulation with predefined cohort size and number of resamples
    #also returns totals_res, extend_res but concatenated over all simulations (N_resamples)
    #Run probabilistic_cohort_outcomes_per_threshold
    #also returns per simulation average ICER, NMB, d_costs, d_qalys
 

In [74]:
import pandas as pd
import numpy as np
import os,sys
sys.path.append(r'C:\Users\hvanvoorst\PhD\git_repos\CLEOPATRA')
sys.path.append(r'C:\Users\henkvanvoorst\Documents\phd\git_repos\CLEOPATRA')
#usefull functions for processing data
from Utils.utils import *
#Objects to initialize for simulations
from Model.Simulate import ControlPatientGeneration, Simulate
from Model.Mortality import Mortality
from Model.RecurrentStroke import RecurrentStroke
from Model.Utilities import Costs, QALYs
#standard simulation functions
from Utils.Experiment import probabilistic_simulation, get_patient_dct, simulate_IDs
#extended analyses functions
from Utils.Experiment import OR_shift_psa, subgroup_psa, M2_miss_psa
from Utils.Outcomes import cohort_outcome
from Utils.Outcomes import probabilistic_cohort_outcomes
import warnings
warnings.filterwarnings('ignore')


p_inputs = r'\model_inputs'

root_sav = r'' #define
root_fig = r'' #define
p_mort_file = os.path.join(p_inputs, 'Forecasted_mortality.xlsx')
p_HR_file = os.path.join(p_inputs, 'Stroke_recurrence.xlsx')
p_cost_file = os.path.join(p_inputs, 'Costs.xlsx')
p_qaly_file = os.path.join(p_inputs, 'QALYs.xlsx')
p_nums_mrs = os.path.join(p_inputs,'mRS_evt_noevt_core_vol.xlsx')
p_OR = os.path.join(p_inputs,'OR_mrcleantrial.xlsx')

CPG = ControlPatientGeneration(
                                p_OR,
                                p_nums_mrs,
                                verbal=False
                                )

M = Mortality(
              p_mort_file = p_mort_file, 
              HR_mrs = np.array([1.54,2.17,3.18,4.55,6.55]),
              years = np.arange(2021,2035)
                )

RS = RecurrentStroke(
                    file_p_HR = p_HR_file,
                    p_mrs_postrestroke = np.array([11/233, 21/233,18/233,22/233,6/233])
                    )

#inflation 2015-2022 to get costs for start reference year 2023
infl_py = [0.6,0.3,1.4,1.7,2.6,1.3,9]
start_infl = np.prod(np.array([1+i/100 for i in infl_py]))

C = Costs(
         file=p_cost_file,
         costs_IVT=950.82, costs_CTP=251.4, costs_EVT=9924.5,
         start_inflation=start_infl, # inflation factor before simulation (2015 -> 2022)
         discounting_rate=1.04,
         inflation_rate=1.017 #future inflation rate
            )

Q = QALYs(
        file = p_qaly_file,
        discounting_rate=1.015
        )

#PM:
# MR CLEAN trial: OR=1.86(CI:1.34-2.59) --> #OR: 1.67; 95% CI, 1.21 to 2.30
# n_evt=233: n_mrs<3 = 33% - 77, n_mrs>2=67% - 156
# n_noevt=267: n_mrs<3 = 19% - 52, n_mrs>2=80% - 214

# HERMES CTP: OR=2.520(CI:1.483-4.283)
# n_evt=289: n_mrs<3 = 136, n_mrs>2: 15

Sim = Simulate(CPG,M,RS,C,Q,5,2023,verbal=False)

df = pd.read_excel(r'/imputed_patient_level_data')
df.ivt_given = df.ivt_given-1
df.index = df.IDs
df['bl_occloc'] = df['bl_occloc'].replace(3,'M1').replace(2,'ICA-T').replace(4,'M2').replace(1,'ICA')
df['age_groups'] = pd.cut(df.r_age,bins=[0,50,60,70,80,1000])
df['otg_groups'] = pd.cut(df.t_otg,bins=[0,120,180,240,300,1000])
df.ivt_given = df.ivt_given-1
df.index = df.IDs
bl_cols = ['core_vol', 'penumbra_vol', 'mm_ratio', 
       'r_age', 'bl_nihss_sum', 't_otg','r_sex',
        'bl_collaterals','bl_hist_premrs','iat_post_etici',
        'age_groups','otg_groups']
BL = df[bl_cols]
bl_dct = BL.to_dict(orient='index')
print(df.shape)

(705, 25)


In [6]:
#baseline simulations
#Sim.verbal = True
totals_res, extend_res = simulate_IDs(df.index,df,Sim)
#add clinical basline info for plots
totals_res = pd.concat([totals_res,BL],axis=1)
#aggr contains average results in NMB, ICER, d_costs,d_qalys
# of the cohort per different decision threshold
out,aggr = cohort_outcome(totals_res,
                        thresholds=np.arange(0,151,10), 
                        costs_per_ctp=0, #used for miss rate sim
                        multiply_ctp_costs = 0,#used for miss rate sim -->perform only on M2
                        miss_percentage = [0],#used for miss rate sim
                        WTP=80000)
#totals_res.to_excel(os.path.join(root_sav,'Baseline','results.xlsx'))
#extend_res.to_excel(os.path.join(root_sav,'Baseline','simulation_output.xlsx'))

#probabilistic standard simulation --> not required since included in OR shift analyses (shift=0)
#extract per patient,strategy, and simulation the results
totals_res, extend_res = probabilistic_simulation(Sim,df,10,100)
#compute difference between arms (PSA computes cohort outcomes per simulation)
outs, aggrs = probabilistic_cohort_outcomes(totals_res,
                                            bl_dct,
                                            thresholds=np.arange(0,151,10),
                                            costs_per_ctp=0,
                                            multiply_ctp_costs = 0,
                                            miss_percentage = [0],
                                            WTP=80000)

100%|████████████████████████████████████████████████████████████████████████████████| 705/705 [00:07<00:00, 94.48it/s]


In [75]:
#probabilistic baseline simulation --> not required since included in OR shift analyses (shift=0)
#extract per patient,strategy, and simulation the results
totals_res, extend_res = probabilistic_simulation(Sim,df,10,100)
#compute difference between arms (PSA computes cohort outcomes per simulation)
outs, aggrs = probabilistic_cohort_outcomes(totals_res,
                                            bl_dct,
                                            thresholds=np.arange(0,151,10),
                                            costs_per_ctp=0,
                                            multiply_ctp_costs = 0,
                                            miss_percentage = [0],
                                            WTP=80000)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.12s/it]


In [ ]:
#Simulates a set of PSAs with different OR values (saves when running)
root_sav = r'\results\PSA\shift_OR'
#baseline PSA is included when shift is set to zero
OR_evt_shifts = [-.1,0] #[-.3,-.2,-.1,0,.1]
OR_evt_corevol_shifts = [-.05,0] #[-.2,-.15,-.1,-.05,0]
ext_res, aggr_res = OR_shift_psa(df,
             OR_evt_shifts,
             OR_evt_corevol_shifts,
             Sim,
             N_resamples=10, #10,000 in protocol --> use 1,000
             N_patients_per_cohort=10, #100 pt in protocol
             thresholds=np.arange(0,151,10),
             seed=21)
if not os.path.exists(root_sav):
    os.makedirs(root_sav)
    
aggr_res.to_excel(os.path.join(root_sav,'aggregated_psa_res.xlsx'))
ext_res.to_excel(os.path.join(root_sav,'extended_psa_res.xlsx'))


In [18]:
#probabilistic subgroup analyses
sgroup_res = subgroup_psa(df,
             col_subgroup,
             Sim,
             N_resamples,
             N_patients_per_cohort,
             thresholds=np.arange(0,151,10),
             costs_per_ctp=0,
             multiply_ctp_costs=0,
             miss_percentage=[0],
             seed=21)

'(70, 80]'